<a href="https://colab.research.google.com/github/tchintchie/whgscraper/blob/master/soup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup as bs
import pandas as pd

In [97]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error(f'Error during requests to {url} : {str(e)}')
        return None

In [98]:

def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

In [99]:

def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [100]:
url = "https://www.oehboersen.at"
html = simple_get(url)

In [101]:
oeh = bs(html, "html5lib")

In [102]:
title = oeh.title.string

In [103]:
rows = oeh.find_all("div", class_="property-content")
rows[0]

<div class="property-content">
		
				<div class="row visible-xs visible-sm">
			<span class="type">2-Zimmer Wohnung</span> <span>·</span> <span class="status">Miete</span>		</div>
		
		<div class="row headlines">
			<a href="https://www.oehboersen.at/property/2-zimmer-wohnung-ueber-den-daechern-der-stadt/" tabindex="0"><h3 class="title col-sm-8">2.Zimmer.Wohnung.Über den Dächern der Stadt</h3> </a>
			<div class="price-tag col-sm-4"><h4 class="price text-primary">€ 597,- <span class="BKInfo">(inkl. BK)</span></h4>
								<h4 class="flaeche">63 m<sup>2</sup></h4>
							</div>
		</div>
		
				
		<div class="row">
			<a href="https://www.oehboersen.at/property/2-zimmer-wohnung-ueber-den-daechern-der-stadt/"><h4 class="address col-sm-8"><strong>Altstadt</strong><br/><br/></h4></a>
			
		</div>
		

			
				<div class="property-meta clearfix">
							<div>
					<div class="meta-title"><i class="fa fa-expand"></i></div>
					<div class="meta-data" data-toggle="tooltip" title="Fläche des Mi

In [109]:
df = pd.DataFrame()
titles = []
types = []
prices = []
bks = []
areas = []
for div in oeh.find_all("div",class_ = "property-content"):
  title = div.find("h3",class_="title col-sm-8").text
  titles.append(title)
  type_ = div.find("span", class_="type").text
  types.append(type_)
  price=div.find("h4",class_= "price text-primary").next
  prices.append(price)
  bk= div.find("span", class_="BKInfo").text
  bks.append(bk)
  if div.find("div", attrs={"title":"Fläche des Mietgegenstandes"}) != None:
    area = div.find("div", attrs={"title":"Fläche des Mietgegenstandes"}).text
  else:
    area = "NA"
  areas.append(area)
  #pd.concat(df, [title, price, bk, area])


In [111]:
print(titles)
print(types)
print(prices)
print(bks)
print(areas)

['2.Zimmer.Wohnung.Über den Dächern der Stadt', 'Gemütliche Bleibe in Sportlicher 3er WG', 'Roomie in 5er WG', 'Schöne 3 Zimmerwohnung in Pradl als 3er WG an StudentIn  ', 'Zwischenmiete August+September, 22qm Zimmer im Saggen', '1-Zi.-Wohnung mit Westbalkon in Innsbruck', '„Zimmer in Veggi-WG ;-) in Arzl“', 'Zimmer in 2er WG fürs Wintersemester 2020/21', 'Helle 2- Zimmerwohnung mit Balkon – Uninähe', '12 qm-Zimmer im grünen Saggen (5er-WG)', 'Helle Garconniere mit Süd-Ausrichtung', 'Untervermietung einer 2-geschossigen Architekten Wohnun  ', 'Zimmer in 2er WG. MIETE', 'Tiefgaragenabstellplatz – Mariahilfpark 2', 'Tiefgaragenplatz', 'Zimmer in 2er WG', 'Schöne 2 Zimmerwohnung mit Garten in Innsbruck. Gelegen  ', 'ZENTRALER PARKPLATZ zu vermieten!', 'Sonnige 3-Zimmer-Wohnung in Kematen, WG-tauglich', '20 m² Zimmer in 3er WG nahe Westbahnhof (Wilten) ab 1.   ', 'Ich biete Privatzimmer mit Balkon, Dusche, Toilette und  ']
['2-Zimmer Wohnung', 'Zimmer in 3er WG', 'Zimmer in 5er WG', '3-Zim

In [106]:
for div in oeh.find_all("div",class_ = "property-content"):
  preis = div.find("h4").text

In [107]:
preis

'€ 400,- (inkl. BK)'

In [108]:
df = pd.DataFrame(price, bk, area)

TypeError: ignored